# Part 1 -- Setup & Collection

Pull Tweets from the Twitter API and collect **all Tweets from 30 tech thought leaders and news outlets**.

### Load lib codes:

In [2]:
!pwd

/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter/ipynb


In [3]:
from os import chdir
chdir('/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter/')

from lib import *
# suppress_warnings()
from lib.twitter_keys import my_keys

In [4]:
!pip install pymongo pyquery tweepy
import pymongo
import tweepy
# from tqdm import tqdm 

In [5]:
# Replace the API_KEY and API_SECRET with your application's key and secret
auth = tweepy.AppAuthHandler(my_keys['CONSUMER_KEY'], my_keys['CONSUMER_SECRET'])

# Authorize twitter, initialize tweepy
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

In [6]:
cli = pymongo.MongoClient(host='35.163.253.8')

In [7]:
# Instantiates when you put data in 
tweets_collection = cli.twitter_db.tweets_collection
cli.twitter_db.collection_names()

[u'completed_collection', u'task_collection']

In [9]:
print(tweets_collection.count())

0


In [10]:
tweets_collection.drop()

In [19]:
chdir('//home/jovyan/work/predicting_stock_market_trends_with_Twitter/tools/GetOldTweets-python-master')
!pip install -r requirements.txt
import got

  Using cached lxml-3.5.0.tar.gz
  Using cached pyquery-1.2.10-py2-none-any.whl
  Running setup.py bdist_wheel for lxml ... - \ | / error
  Complete output from command /opt/conda/envs/python2/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-build-7O91Dc/lxml/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/tmp3Ck1SBpip-wheel- --python-tag cp27:
  Building lxml version 3.5.0.
  Building without Cython.
  ERROR: /bin/sh: 1: xslt-config: not found
  
  ** make sure the development packages of libxml2 and libxslt are installed **
  
  Using build configuration of libxslt
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-2.7
  creating build/lib.linux-x86_64-2.7/lxml
  copying src/lxml/__init__.py -> build/lib.linux-x86_64-2.7/lxml
  copying src/lxml/usedoctest.py -> build/lib.linux-x86_64-2.7/lxml
  

### Implement distributed processing from scratch - run this on multiple AWS instances:

In [20]:
twitter_30_df = pd.read_csv('/home/jovyan/work/predicting_stock_market_trends_with_Twitter/data/twitter_users_30.csv')
twitter_30_df.sample(3)

,Name,Handle,Followers,Description,Type,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
14,Forbes Tech News,ForbesTech,"2,600,000",Tech news and insights from @Forbes.,News Outlet,NaN,NaN,NaN,NaN
26,Mark Lehman,markflowchatter,"33,400",has more than 23 years of experience in the ma...,Thought Leader,NaN,NaN,NaN,NaN
19,Downtown Josh Brown,ReformedBroker,"663,000","Chairman of the Twitter Federal Reserve, star ...",Thought Leader,NaN,NaN,NaN,NaN


In [24]:
handle = list(twitter_30_df['Handle'])

In [25]:
def get_tweets_to_mongo(lookup):

    maxTweets = 10000000 # Some arbitrary large number
    tweetsPerQry = 200  # this is the max the API permits

    for l in lookup:
        # If results from a specific ID onwards are reqd, set since_id to that ID.
        # else default to no lower limit, go as far back as API allows
        sinceId = None

        # If results only below a specific ID are, set max_id to that ID.
        # else default to no upper limit, start from the most recent tweet matching the search query.
        max_id = -1E10

        tweetCount = 0
        while tweetCount < maxTweets:
            try:
                if (max_id <= 0):
                    if (not sinceId):
                        new_tweets = api.user_timeline(screen_name = l, count=tweetsPerQry)
                    else:
                        new_tweets = api.user_timeline(screen_name = l, count=tweetsPerQry,
                                                    since_id=sinceId)
                else:

                    if (not sinceId):
                        new_tweets = api.user_timeline(screen_name = l, count=tweetsPerQry,
                                                    max_id=str(max_id - 1))
                    else:
                        new_tweets = api.user_timeline(screen_name = l, count=tweetsPerQry,
                                                    max_id=str(max_id - 1),
                                                    since_id=sinceId)
                if not new_tweets:
                    print("No more tweets found")
                    break
                for tweet in new_tweets:
                    tweet_dict = {'username': tweet.user.screen_name,
                                  'timestamp': tweet.created_at, 
                                  'text': tweet.text.encode("utf-8")}
                    
                    mongo_response = tweets_collection.insert_one(tweet_dict)

                tweetCount += len(new_tweets)
                print("Downloaded {0} tweets for user: {1}".format(tweetCount, l))
                max_id = new_tweets[-1].id  

            except tweepy.TweepError as e:
                # Just exit if any error
                print("some error : " + str(e))
                break

        print ("Downloaded {0} tweets for user: {1} & saved to Mongo\n".format(tweetCount, l))

In [26]:
from datetime import datetime
start = datetime.now()
get_tweets_to_mongo(handle)
end = datetime.now()
print(end - start)

Downloaded 200 tweets for user: BarackObama
Downloaded 400 tweets for user: BarackObama
Downloaded 600 tweets for user: BarackObama
Downloaded 800 tweets for user: BarackObama
Downloaded 1000 tweets for user: BarackObama
Downloaded 1200 tweets for user: BarackObama
Downloaded 1400 tweets for user: BarackObama
Downloaded 1600 tweets for user: BarackObama
Downloaded 1800 tweets for user: BarackObama
Downloaded 2000 tweets for user: BarackObama
Downloaded 2200 tweets for user: BarackObama
Downloaded 2399 tweets for user: BarackObama
Downloaded 2599 tweets for user: BarackObama
Downloaded 2796 tweets for user: BarackObama
Downloaded 2996 tweets for user: BarackObama
Downloaded 3196 tweets for user: BarackObama
Downloaded 3234 tweets for user: BarackObama
No more tweets found
Downloaded 3234 tweets for user: BarackObama & saved to Mongo

Downloaded 200 tweets for user: cnnbrk
Downloaded 400 tweets for user: cnnbrk
Downloaded 600 tweets for user: cnnbrk
Downloaded 800 tweets for user: cnnbrk

In [28]:
tweets_collection.count()

87203

In [36]:
87203.0/90000.0

0.9689222222222222

In [ ]:
curs = tweets_collection.find()
list_of_docs = []
for i in range(curs.count()):
    list_of_docs.append(curs.next())
    df = pd.DataFrame(list_of_docs)
df

In [ ]:
df.shape